# Dogs vs Cats
## Kaggle Dataset의 전부를 이용한 개, 고양이 구분
### Dog Image: 12,500개, Cat Image: 12,500개, 총 25,000개
### 출처: [pontoregende GitHub](https://github.com/pontorezende/Dogs-vs-Cats-Redux-with-CNN)

In [40]:
import numpy as np
from glob import glob
import cv2, os, random
import matplotlib.pyplot as plt
%matplotlib inline

In [41]:
import tensorflow as tf
from tensorflow.keras.preprocessing import image
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D
from tensorflow.keras.layers import Dense, Flatten, Dropout
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import ModelCheckpoint
from keras.callbacks import ModelCheckpoint, EarlyStopping

Using TensorFlow backend.


In [42]:
path='dogs-vs-cats/train'

In [43]:
seed = 2020
np.random.seed(seed)
tf.random.set_seed(seed)

In [44]:
## used for resize and in our model
ROW, COL = 96, 96

dogs, cats = [], []
y_dogs, y_cats = [], []

In [45]:
## Definition to load all our dog images
def load_dogs():
    print('Loading all dog images\n')
    dog_path = os.path.join(path, 'dog*')
    for dog_img in glob(dog_path):
        dog = cv2.imread(dog_img)
        dog = cv2.cvtColor(dog, cv2.COLOR_BGR2GRAY)
        dog = cv2.resize(dog, (ROW, COL))
        dog = image.img_to_array(dog)
        dogs.append(dog)
    print('All dog images loaded')

In [46]:
## Definition to load all our cat images
def load_cats():
    print('Loading all cat images\n')
    cat_path = os.path.join(path, 'cat*')
    for cat_img in glob(cat_path):
        cat = cv2.imread(cat_img)
        cat = cv2.cvtColor(cat, cv2.COLOR_BGR2GRAY)
        cat = cv2.resize(cat, (ROW, COL))
        cat = image.img_to_array(cat)
        cats.append(cat)
    print('All cat images loaded')

In [47]:
load_dogs()

Loading all dog images

All dog images loaded


In [48]:
load_cats()

Loading all cat images

All cat images loaded


In [49]:
classes = ['dog', 'cat']

In [50]:
## in case we want to see if our images was saved correctly in arrays we can use those codes
def show_dogs():
    plt.figure(figsize=(12,8))    
    for i in range(5):
        plt.subplot(1, 5, i+1)
        img = image.array_to_img(random.choice(dogs))
        plt.imshow(img)
        
        plt.axis('off')
        plt.title('Supposed to be a {}'.format(classes[0]))        
    plt.show()

In [51]:
def show_cats():
    plt.figure(figsize=(12,8))
    for i in range(5):
        plt.subplot(1, 5, i+1)
        img = image.array_to_img(random.choice(cats))
        plt.imshow(img)

        plt.axis('off')
        plt.title('Supposed to be a {}'.format(classes[1]))
    plt.show()

In [52]:
## just change the labels for 0 and  1
y_dogs = [1 for item in enumerate(dogs)]
y_cats = [0 for item in enumerate(cats)]

In [53]:
## converting everything to Numpy array to fit in our model
## them creating a X and target file like we used to see
## in Machine and Deep Learning models
dogs = np.asarray(dogs).astype('float32')
cats = np.asarray(cats).astype('float32')
y_dogs = np.asarray(y_dogs).astype('int32')
y_cats = np.asarray(y_cats).astype('int32')

In [54]:
## fit values between 0 and 1
dogs /= 255
cats /= 255

In [55]:
X = np.concatenate((dogs,cats), axis=0)
y = np.concatenate((y_dogs, y_cats), axis=0)

In [56]:
IMG_CHANNEL = 1
BATCH_SIZE = 128
N_EPOCH = 10
VERBOSE = 1
VALIDAION_SPLIT = .2
OPTIM = Adam()
N_CLASSES = 2

In [57]:
## One-Hot Encoding - train
y = tf.keras.utils.to_categorical(y, N_CLASSES)

In [58]:
from sklearn.model_selection import train_test_split

X_train, X_test, Y_train, Y_test = train_test_split(X, y, 
                                                    test_size=.2, random_state=seed)

In [59]:
len(X_train)

20000

In [60]:
X_train.shape

(20000, 96, 96, 1)

In [61]:
len(X_test)

5000

In [62]:
X_test.shape

(5000, 96, 96, 1)

In [63]:
Y_train.shape

(20000, 2)

In [64]:
Y_test.shape

(5000, 2)

In [65]:
## Here is our model as a CNN
model = Sequential([
    Conv2D(32, (3,3), padding='same', input_shape=(ROW, COL, IMG_CHANNEL), activation='relu'),
    Conv2D(32, (3,3), padding='same', activation='relu'),
    MaxPooling2D(pool_size=(2,2)),
    Dropout(.25),
    Conv2D(64, (3,3), padding='same', activation='relu'),
    Conv2D(64, (3,3), padding='same', activation='relu'),
    MaxPooling2D(pool_size=(2,2)),
    Dropout(.25),
    Flatten(),
    Dense(512, activation='relu'),
    Dropout(.5),
    Dense(N_CLASSES, activation='softmax')
])
model.summary()

Model: "sequential_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_12 (Conv2D)           (None, 96, 96, 32)        320       
_________________________________________________________________
conv2d_13 (Conv2D)           (None, 96, 96, 32)        9248      
_________________________________________________________________
max_pooling2d_6 (MaxPooling2 (None, 48, 48, 32)        0         
_________________________________________________________________
dropout_9 (Dropout)          (None, 48, 48, 32)        0         
_________________________________________________________________
conv2d_14 (Conv2D)           (None, 48, 48, 64)        18496     
_________________________________________________________________
conv2d_15 (Conv2D)           (None, 48, 48, 64)        36928     
_________________________________________________________________
max_pooling2d_7 (MaxPooling2 (None, 24, 24, 64)       

In [66]:
model.compile(loss='categorical_crossentropy', optimizer=OPTIM, metrics=['accuracy'])

In [69]:
modelpath = "model/dogs_vs_cats-cnn.hdf5"
checkpointer = ModelCheckpoint(filepath=modelpath, monitor='val_loss', 
                               verbose=VERBOSE, save_best_only=True)
early_stopping_callback = EarlyStopping(monitor='val_loss', patience=20)

In [70]:
model.fit(X_train, Y_train, batch_size=BATCH_SIZE, epochs=N_EPOCH, validation_split=VALIDAION_SPLIT,shuffle=True,callbacks=[checkpointer, early_stopping_callback])

Train on 16000 samples, validate on 4000 samples
Epoch 1/10
15872/16000 [============================>.] - ETA: 1s - loss: 0.6887 - accuracy: 0.5374
Epoch 00001: val_loss improved from inf to 0.66773, saving model to model/dogs_vs_cats-cnn.hdf5
16000/16000 [==============================] - 212s 13ms/sample - loss: 0.6884 - accuracy: 0.5383 - val_loss: 0.6677 - val_accuracy: 0.5880
Epoch 2/10
15872/16000 [============================>.] - ETA: 1s - loss: 0.6412 - accuracy: 0.6366
Epoch 00002: val_loss improved from 0.66773 to 0.60539, saving model to model/dogs_vs_cats-cnn.hdf5
16000/16000 [==============================] - 220s 14ms/sample - loss: 0.6409 - accuracy: 0.6371 - val_loss: 0.6054 - val_accuracy: 0.6660
Epoch 3/10
15872/16000 [============================>.] - ETA: 1s - loss: 0.5735 - accuracy: 0.7038
Epoch 00003: val_loss improved from 0.60539 to 0.54596, saving model to model/dogs_vs_cats-cnn.hdf5
16000/16000 [==============================] - 220s 14ms/sample - loss: 0.5

In [75]:
from tensorflow.keras.models import load_model

model = load_model('model/dogs_vs_cats-cnn.hdf5')

In [76]:
scores = model.evaluate(X_test, Y_test, verbose=2)
print('MODEL ACCURACY: %.5f' % scores[1])

5000/5000 - 11s - loss: 0.4549 - accuracy: 0.7878
MODEL ACCURACY: 0.78780
